# Cryptocurrency Price Prediction
## Machine Learning Modeling Notebook

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning Libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

## 1. Data Loading and Preprocessing

In [ ]:
# Load processed cryptocurrency data
def load_crypto_data(crypto_name):
    """
    Load and preprocess cryptocurrency data
    
    Args:
        crypto_name (str): Name of the cryptocurrency
    
    Returns:
        Processed dataset ready for modeling
    """
    # Load data from processed directory
    data_path = f'../data/processed/{crypto_name}_processed.csv'
    df = pd.read_csv(data_path)
    
    # Convert timestamp
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    return df

## 2. Feature Engineering

In [ ]:
def create_time_series_features(df):
    """
    Create additional time series features
    
    Args:
        df (pd.DataFrame): Input dataframe
    
    Returns:
        DataFrame with additional features
    """
    # Rolling window features
    df['rolling_mean_7d'] = df['price'].rolling(window=7).mean()
    df['rolling_std_7d'] = df['price'].rolling(window=7).std()
    
    # Price changes
    df['price_change_1d'] = df['price'].pct_change()
    df['price_change_7d'] = df['price'].pct_change(periods=7)
    
    # Lagged features
    df['price_lag_1'] = df['price'].shift(1)
    df['price_lag_7'] = df['price'].shift(7)
    
    return df.dropna()

## 3. Sequence Preparation for LSTM

In [ ]:
def create_sequences(data, seq_length=30):
    """
    Create sequences for LSTM modeling
    
    Args:
        data (np.array): Input data
        seq_length (int): Length of input sequences
    
    Returns:
        X, y sequences for LSTM
    """
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    
    return np.array(X), np.array(y)

## 4. LSTM Model Architecture

In [ ]:
def build_lstm_model(input_shape):
    """
    Build LSTM Neural Network for Price Prediction
    
    Args:
        input_shape (tuple): Shape of input sequences
    
    Returns:
        Compiled Keras LSTM model
    """
    model = Sequential([
        # First LSTM Layer
        LSTM(units=50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        
        # Second LSTM Layer
        LSTM(units=50, return_sequences=False),
        Dropout(0.2),
        
        # Dense Layers
        Dense(25, activation='relu'),
        Dense(1)  # Output layer
    ])
    
    # Compile Model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mean_squared_error',
        metrics=['mae']
    )
    
    return model

## 5. Model Training and Evaluation

In [ ]:
def train_and_evaluate_model(crypto_name):
    """
    Train and evaluate LSTM model for cryptocurrency
    
    Args:
        crypto_name (str): Name of cryptocurrency
    
    Returns:
        Trained model and performance metrics
    """
    # Load and preprocess data
    df = load_crypto_data(crypto_name)
    df = create_time_series_features(df)
    
    # Scale features
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df[['price']])
    
    # Create sequences
    X, y = create_sequences(scaled_data)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=False
    )
    
    # Build model
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2])
    )
    
    # Early stopping
    early_stop = EarlyStopping(
        monitor='val_loss', 
        patience=10, 
        restore_best_weights=True
    )
    
    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stop],
        verbose=1
    )
    
    # Predictions
    y_pred = model.predict(X_test)
    
    # Inverse transform predictions
    y_test_inv = scaler.inverse_transform(y_test)
    y_pred_inv = scaler.inverse_transform(y_pred)
    
    # Performance metrics
    mse = mean_squared_error(y_test_inv, y_pred_inv)
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    r2 = r2_score(y_test_inv, y_pred_inv)
    
    # Visualization
    plt.figure(figsize=(12, 6))
    plt.plot(y_test_inv, label='Actual Prices', color='blue')
    plt.plot(y_pred_inv, label='Predicted Prices', color='red')
    plt.title(f'{crypto_name.capitalize()} Price Prediction')
    plt.legend()
    plt.savefig(f'../outputs/{crypto_name}_prediction.png')
    plt.close()
    
    return model, {
        'MSE': mse,
        'MAE': mae,
        'R2': r2
    }

## 6. Main Execution

In [ ]:
def main():
    # List of cryptocurrencies to analyze
    cryptocurrencies = ['bitcoin', 'ethereum', 'binancecoin']
    
    # Store results
    model_results = {}
    
    # Train models for each cryptocurrency
    for crypto in cryptocurrencies:
        print(f"Training model for {crypto}...")
        model, metrics = train_and_evaluate_model(crypto)
        
        # Save model
        model.save(f'../models/{crypto}_lstm_model.h5')
        
        # Store results
        model_results[crypto] = metrics
    
    # Print results
    for crypto, metrics in model_results.items():
        print(f"\n{crypto.capitalize()} Model Performance:")
        for metric, value in metrics.items():
            print(f"{metric}: {value}")

if __name__ == '__main__':
    main()